# 导入所需的模块

In [ ]:
from selenium import webdriver
import re
import codecs
from selenium.webdriver.common.keys import Keys
import time
import requests
import os

# 创建所需的列表

In [ ]:
def get_list():
    lj = []
    py = []
    data = []
    wenzi = []

# 创建数据库

In [ ]:
from pony.orm import *


db = Database()


class Hanzi(db.Entity):
    id = PrimaryKey(int, auto=True)
    Chinese = Optional(str,column ='Chinese')
    spell = Optional(str,column ='spell')
    link = Optional(str,column ='link')



# db.generate_mapping()

# 构建创建数据库的函数

In [ ]:
def get_database():
    dbpath = 'f:/ex9/hanzi.sqlite'
    if os.path.exists(dbpath):
        os.remove(dbpath)
    f = open(dbpath,"w")
    f.close()
    db.bind(provider='sqlite',filename='f:/ex9/hanzi.sqlite')
    db.generate_mapping(create_tables=True)
    set_sql_debug(True)    

# 创建查找汉字拼音的函数

In [ ]:
def get_spell(h):
    reobj = re.compile(r'<span class="op_exactqa_detail_word_pronounce">([\d\D]*?)<')
    for match in reobj.finditer(h):
        a = match.group(1)
        py.append(a)

# 创建查找汉字发音链接的函数

In [ ]:
def get_link(h):
    reobj = re.compile(r"""data-click="{fm:'beha'}" url="([\d\D]*?)" h""")
    for match in reobj.finditer(h):
        a = match.group(1)
        lj.append(a)

# 获取页面html,并且获取汉字的发音链接和拼音

In [ ]:
def get_data(a):
    url="https://www.baidu.com/s?wd=" + a + "拼音"
    head = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36',
           'Cookie': 'BIDUPSID=D7397E6837AC5F69C2D793409730A5E7; PSTM=1581844331; BAIDUID=D7397E6837AC5F690B98E2A4ED13E490:FG=1; sug=3; sugstore=1; ORIGIN=0; bdime=0; BD_UPN=12314753; BDORZ=B490B5EBF6F3CD402E515D22BCDA1598; COOKIE_SESSION=1357348_0_3_2_0_1_0_0_2_1_0_0_0_0_0_0_0_0_1589767631%7C3%230_0_1589767631%7C1; BD_HOME=1; H_PS_PSSID=1428_31325_21091_31110_31595_31270_31463_31322_30823_26350; BDRCVFR[feWj1Vr5u3D]=mk3SLVN4HKm; delPer=0; BD_CK_SAM=1; PSINO=3; H_PS_645EC=9ab6WX6dqMvjKhk3hVBc%2Fr05dYm%2FjpVHIpbZERUZtetCpfPY%2FFB52x4kgwKBh0RS%2FMUN'}
    r = requests.get(url,headers = head)
    r.keep_alive = False
    r.encoding = 'utf-8'
    h = r.text
    
    get_spell(h)
    get_link(h)
    
    for i in range(len(lj)):
        data.append(( a , py[i] , lj[i] ))

# 创建打开txt文件，将文件中的字存入列表wenzi中的函数

In [ ]:
def get_hanzi():
    f = codecs.open(r"C:\Users\19148\Desktop\初始.txt","r","utf-8-sig")
    for hz in f.readlines():
        a = hz.replace('\r','').replace('\n','')
        wenzi.append(a)
    f.close()

# 创建将文字存入csv文件的函数

In [ ]:
def get_csv():
    f = codecs.open(r"f:\999.csv","w","utf_8_sig")
    f.write("汉字,拼音,拼音发声的链接\n")
    for data1 in data:
        f.write(data1[0] + "," + data1[1] + "," + data1[2] + "\n")
    f.close()

# 清空列表lj，py的内容

In [ ]:
def get_clear():
    lj.clear()
    py.clear()
    data.clear()
    wenzi.clear()

# 任务一：利用百度的汉字查找拼音功能，以元组形式输出出某个汉字对应的以下内容：（该汉字，拼音，拼音发声的链接）

In [ ]:
get_list()

In [ ]:
get_data("任")
data

In [ ]:
get_clear()

# 任务二：在任务1的基础上进行延展，已知一个txt文本文件每行只有一个文字,从该文件逐行读取文字，并构建任务1的链接，抽取拼音及发音链接，构建一个CSV文件.

In [ ]:
get_list()

In [ ]:
get_hanzi()
for a in wenzi:
    lj.clear()
    py.clear()
    get_data(a)
get_csv()

In [ ]:
get_clear()

# 任务三：存储到数据库

In [ ]:
get_list()

In [ ]:
get_database()

In [ ]:
get_hanzi()
for a in wenzi:
    lj.clear()
    py.clear()
    get_data(a)
for data1 in data:
    s=Hanzi(Chinese = data1[0],spell = data1[1],link = data1[2])
    db.commit()